## Transformations in Pinocchio
In this tutorial we will introduce how to perform rigid body transformations using the pinocchio library. We will go through the following three exercises:
1. Global and Relative Transformations
2. Exponential Coordinate Representation of Rigid Body Transforms
3. Twist and Wrench Transforms 

Before we complete the three exercises, we will first perform the setup and demonstrate how to display objects using meshcat.

## Set up
We will need NumPy, Pinocchio, and MeshCat Viewer for vizualizing the robot.

In [1]:
import time
import numpy as np
import pinocchio as pin
from utils.meshcat_viewer_wrapper import MeshcatVisualizer

<a id='section_display_objects'></a>
## Displaying objects
Let's first learn how to open a 3D viewer, in which we will build our simulator. We will use the viewer MeshCat which directly displays in a browser. Open it as follows:

In [2]:
viz = MeshcatVisualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


The viz object is a client of the viewer, i.e. it will be use to pass display command to the viewer. The first commands are to create objects:

In [3]:
ballID = 'world/ball'; viz.addSphere(ballID,.2,[1,0,0,1])
boxID = 'world/box';   viz.addBox(boxID,[.5,.2,.4],[1,1,0,1])
cylID1 = 'world/cyl1';   viz.addCylinder(cylID1,length=0.3,radius=.05,color=[0,0,1,1])
cylID2 = 'world/cyl2';   viz.addCylinder(cylID2,length=0.2,radius=.06,color=[1,1,0,1])

In [4]:
viz.delete(ballID)
viz.delete(boxID)

Lets also visualize four random coordinate frames and transform the objects to the frames.

In [5]:
frame1 = pin.SE3.Random()
frame2 = pin.SE3.Random()
frame3 = pin.SE3.Random()
frame4 = pin.SE3.Random()

viz.visualize_frame("frame1", frame1)
viz.visualize_frame("frame2", frame2)
viz.visualize_frame("frame3", frame3)
viz.visualize_frame("frame4", frame4)

Run the next cell to transform the two cylinders to each frame in meshcat.

In [6]:
for frame in [frame1, frame2, frame3, frame4]:
    viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(frame))
    viz.applyConfiguration(cylID2,pin.SE3ToXYZQUAT(frame))
    time.sleep(0.5)
viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(pin.SE3(1)))
viz.applyConfiguration(cylID2,pin.SE3ToXYZQUAT(pin.SE3(1)))

In [7]:
viz.delete("frame1")
viz.delete("frame2")
viz.delete("frame3")
viz.delete("frame4")

Now that we have seen how to define transformations in pinocchio and how to visualize them in meshcat, lets move onto the first exercise.

### Exercise 1
Given the following relative transformations T_10, T_12, T_23, T43. Find the global transformations T_01, T_02, T_03, T_04 and transform the two cylinders to each frame. Visualize all of the frames and perform the subsequent global transformations to both cylinders. Then run the test cell to make sure that your computed transforms are correct.

The transform notation corresponds to the one used in *Modern Robotics*.
$$T_{01} = \begin{bmatrix} 
    R_{01} & p_{01} \\
    0 & 1 
\end{bmatrix}$$

In [8]:
R_10 = np.array([[0.180678, 0.950891, -0.251321],
                 [0.89108, -0.266422, -0.367417],
                 [-0.416331, -0.157563, -0.895457]])
p_10 = np.array([0.740223, 0.593148, -0.640237])
T_10 = pin.SE3(R_10, p_10)

R_12 = np.array([[-0.246977, 0.817821, -0.51978],
                 [0.963818, 0.15181, -0.219108],
                 [-0.100284, -0.555088, -0.825724]])
p_12 = np.array([1.55288, 0.90413, -0.169165])
T_12 = pin.SE3(R_12, p_12)

R_23 = np.array([[-0.49966, -0.519852, 0.692888],
                 [-0.158395, 0.841243, 0.516935],
                 [-0.851617, 0.148542, -0.502677]])
p_23 = np.array([-0.150189, -1.25202, 0.767956])
T_23 = pin.SE3(R_23, p_23)

R_43 = np.array([[0.420593, 0.383923, 0.822013],
                 [-0.882899, 0.381713, 0.273467],
                 [-0.208782, -0.840773, 0.499511]])
p_43 = np.array([0.293482, -0.372091, 0.23161])
T_43 = pin.SE3(R_43, p_43)

In [9]:
T_01 = T_10.inverse()
T_02 = T_01 * T_12
T_03 = T_01 * T_12 * T_23
T_04 = T_01 * T_12 * T_23 * T_43.inverse()

In [10]:
viz.visualize_frame("T_01", T_01)
viz.visualize_frame("T_02", T_02)
viz.visualize_frame("T_03", T_03)
viz.visualize_frame("T_04", T_04)

In [11]:
for frame in [T_01, T_02, T_03, T_04]:
    viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(frame))
    viz.applyConfiguration(cylID2,pin.SE3ToXYZQUAT(frame))
    time.sleep(1)
viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(pin.SE3(1)))
viz.applyConfiguration(cylID2,pin.SE3ToXYZQUAT(pin.SE3(1)))

Fill in the missing parts of the transform_error function to test your transformations against the solutions. An error on the order of 1e-7 is expected for correct transformations.

In [12]:
from sol.pin_transformations import T_01_sol, T_02_sol, T_03_sol, T_04_sol

def transform_error(T, T_sol):
    SE3_error = pin.log(T_sol.inverse() * T)
    scalar_error = np.linalg.norm(SE3_error.linear + SE3_error.angular)
    return scalar_error

print(transform_error(T_01, T_01_sol))
print(transform_error(T_02, T_02_sol))
print(transform_error(T_03, T_03_sol))
print(transform_error(T_04, T_04_sol))

6.14095854715443e-07
3.5250915839765295e-07
7.520391730676697e-07
6.745416992167283e-07


In [13]:
viz.delete("T_01")
viz.delete("T_02")
viz.delete("T_03")
viz.delete("T_04")
viz.delete(cylID1)
viz.delete(cylID2)

### Exercise 2
In this exercise we will visualize the screw interpretation of a rigid body transformation. Through this we will get practice with the exponential coordinate representation of rigid body transformations.

First lets get familiar with the pinocchio implementations of the log and exp functions that we learned about in the lecture.

$$\begin{aligned}
    \text{log}: &\quad T \in SE(3) \quad \rightarrow \quad {\mathcal{V}} = [S]\theta \in se(3).
\end{aligned}$$

In [14]:
print("Rigid Body Transform in SE(3):")
print(T_01)
print("Twist V in se(3) associated with the Rigid Body Transform in SE(3):")
V_01 = pin.log(T_01)
print(V_01)
print(V_01.linear)
print(V_01.angular)

Rigid Body Transform in SE(3):
  R =
 0.180678   0.89108 -0.416331
 0.950891 -0.266422 -0.157563
-0.251321 -0.367417 -0.895457
  p = -0.928835 -0.646721 -0.169338

Twist V in se(3) associated with the Rigid Body Transform in SE(3):
  v =   -1.1947 -0.142939  0.287799
  w = -2.30461 -1.81213 0.656842

[-1.19470225 -0.14293948  0.28779915]
[-2.30460792 -1.81213297  0.65684192]


$$\begin{aligned}
    \text{log}: &\quad R \in SO(3) \quad \rightarrow \quad \omega = [\hat{\omega}]\theta \in so(3).
\end{aligned}$$

In [15]:
print("Matrix log of SO(3)")
omega = pin.log3(T_01.rotation)
print("omega =", omega,"\n")

Matrix log of SO(3)
omega = [-2.30460792 -1.81213297  0.65684192] 



Recovering $\hat{\omega}$ and $\theta$ can be done using the algorithm from section 3.2.3.3 of *Modern Robotics*. Take a look at it if you would like to know the details. In pinocchio we can simply call pin.AngleAxis.

In [16]:
print("Angle axis representation of an SO(3) element")
print(pin.AngleAxis(T_01.rotation))
print("Recover axis from omega")
print("omega_hat =", omega / pin.AngleAxis(T_01.rotation).angle)

Angle axis representation of an SO(3) element
angle: 3.00437
axis: -0.767075 -0.603157  0.218624

Recover axis from omega
omega_hat = [-0.76708421 -0.60316489  0.21862854]


$$\begin{aligned}
    \text{exp}: &\quad {\mathcal{V}} = [S]\theta \in se(3) \quad \rightarrow \quad T \in SE(3)
\end{aligned}$$

In [18]:
print("Matrix exponential for se(3):")
print(pin.exp(V_01))

Matrix exponential for se(3):
  R =
 0.180674   0.89109 -0.416312
 0.950885 -0.266425 -0.157594
-0.251346 -0.367392  -0.89546
  p = -0.928848 -0.646741 -0.169335



$$\begin{aligned}
    \text{exp}: &\quad \omega = [\hat{\omega}]\theta \in so(3) \quad \rightarrow \quad R \in SO(3)
\end{aligned}$$

In [19]:
print("Matrix exponential for so(3):")
print(pin.exp3(omega))
print("Compare to original rotation matrix:")
print(T_01.rotation)

Matrix exponential for so(3):
[[ 0.18067434  0.89108975 -0.4163122 ]
 [ 0.95088475 -0.26642516 -0.15759389]
 [-0.25134634 -0.36739175 -0.89545984]]
Compare to original rotation matrix:
[[ 0.180678  0.89108  -0.416331]
 [ 0.950891 -0.266422 -0.157563]
 [-0.251321 -0.367417 -0.895457]]


Now that we have presented the necessary pinocchio functions, let's move on to the exercise.

The goal of this exercise is to visualize a rigid body transformation using the screw interpretation of a twist. Fill in the missing parts of the code below. Here the function screw_axis_q() should return the position q of the axis in space and distance d travelled on the screw axis and s_hat the unit vector representing the direction of the screw axis. This equation is found in section 3.3.2.2 in *Modern Robotics*:  
$${\mathcal{V}} = \begin{bmatrix}
\omega \\
v
\end{bmatrix} = \begin{bmatrix}
\hat{s} \dot{\theta} \\
-\hat{s} \dot{\theta} \times q + h \hat{s} \dot{\theta}
\end{bmatrix} \\\text{where}:\\
\dot{\theta} = \lVert \omega \rVert\\
\hat{s} = \omega / \lVert \omega \rVert\\
h = \hat{\omega}^Tv/\dot{\theta}$$

hints:  
see relevant section in book for detailed explanation.  
$\text{pin.skew}(\omega) = [\omega]$

In [20]:
def screw_axis_q(se3_twist):
    """args:
    se3_twist: pin se3 object
    returns:
    q: 3x1 np array
    d: float
    """
    s_hat = se3_twist.angular / np.linalg.norm(se3_twist.angular)
    theta_dot = np.linalg.norm(se3_twist.angular)
    h = s_hat.T @ se3_twist.linear / theta_dot

    q = np.linalg.pinv(pin.skew(se3_twist.angular)) @ \
        (h * se3_twist.angular - se3_twist.linear)
    d = h * theta_dot
    return q, d, s_hat

In [21]:
cylID1 = 'world/cyl1';   viz.addCylinder(cylID1,length=0.3,radius=.05,color=[0,0,1,1])
viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(pin.SE3(1)))

First lets test the visualization on a simple example.

In [22]:
transform = pin.SE3(pin.utils.rotate('x',np.pi), np.array([1,0,0]))
viz.visualize_frame("transform", transform)

V = pin.log(transform)
q, d, s_hat = screw_axis_q(V)

viz.visualize_axis("screw_axis", q, s_hat, d)

steps = 200
for i in range(steps + 1):
    alpha = i / steps
    # Interpolate using the exponential map of the weighted twist
    Ti = pin.exp(alpha * V)
    viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(Ti))
    time.sleep(0.01)

See if your transform is the same as the one shown in the video below.

In [23]:
from IPython.display import Video

video_path = "videos/simple_screw_transform.mp4"
Video(video_path)

In [24]:
viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(pin.SE3(1)))
viz.delete("transform")
viz.delete("screw_axis")

Now find visualize the screw transformation on two consecutive transformations. We will use T_01 and T_02 from the previous exercise.

In [25]:
viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(pin.SE3(1)))
viz.visualize_frame("T_01", T_01)
viz.visualize_frame("T_02", T_02)
steps = 200

V = pin.log(T_01)
q, d, s_hat = screw_axis_q(V)
viz.visualize_axis("screw_axis", q, s_hat, d)

for i in range(steps + 1):
    alpha = i / steps
    # Interpolate using the exponential map of the weighted twist
    Ti = pin.exp(alpha * V)
    viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(Ti))
    time.sleep(0.01)

Fill in the missing code and run to see the relative transform.

In [26]:
T_12 = T_01.inverse() * T_02
steps = 200

V_12 = pin.log(T_12)
q_1, d, s_hat_1 = screw_axis_q(V_12)
# Transform q_1 and s_hat_1 to the global frame 0
q_0 = (T_01.homogeneous @ np.append(q_1,[1]))[:3]
s_hat_0 = T_01.rotation @ s_hat_1
viz.visualize_axis("screw_axis", q_0, s_hat_0, d)

for i in range(steps + 1):
    alpha = i / steps
    # Interpolate using the exponential map of the weighted twist
    Ti_12 = pin.exp(alpha * V_12)
    viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(T_01 * Ti_12))
    time.sleep(0.01)

In [27]:
viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(pin.SE3(1)))
viz.delete("T_01")
viz.delete("T_02")
viz.delete("screw_axis")
viz.delete(cylID1)

## Exercise 3
In this exercise we will use pinocchio to transform twists and wrenches between different coordinate frames. Check out the next cell to see how twists and wrenches are defined in pinocchio. Then complete the missing code in the cell with the adjoint_from_SE3() function and manually perform the change of coordinate for both the twist and the wrench. Compare your results with what is outputted in the cell below.

In [28]:
# Twist in Global Frame
V_0 = pin.Motion(np.array([1,2,3]), np.array([1,-1,1]))
print(T_01 * V_0)
# Wrench in Global Frame
W_0 = pin.Force(np.array([1,2,3]), np.array([1,-1,1]))
print(T_01 * W_0)

  v =  1.39733 -0.58774 -5.38554
  w =  -1.12673   1.05975 -0.779361

  f =  0.713845 -0.054642  -3.67253
tau =   1.23912   -2.4723 -0.266949



#### Twist change of coordinate frame
$${\mathcal{V}_a} = [\text{Ad}_{T_{ab}}]{\mathcal{V}_b}$$

#### Wrench change of coordinate frame
$${\mathcal{F}_a} = [\text{Ad}_{T_{ba}}]^T{\mathcal{F}_b}$$

Hint for wrench transform:
$$[\text{Ad}_T]^{-1} = [\text{Ad}_{T^{-1}}]$$

In [29]:
def adjoint_from_SE3(SE3):
    """"
    input: pinocchio SE3 object
    output: 6x6 numpy array
    """
    Adj = np.block([[SE3.rotation, np.zeros((3,3))],
              [pin.skew(SE3.translation) @ SE3.rotation, SE3.rotation]])
    return Adj

Compare the your transformation with that done above with pinocchio

In [30]:
Adj_T_01 = adjoint_from_SE3(T_01)
print("Twist in global frame")
print(Adj_T_01 @ np.hstack((V_0.angular, V_0.linear)))
Adj_T_10 = adjoint_from_SE3(T_01.inverse())
print("Wrench in global frame")
print(Adj_T_10.T @ np.hstack((W_0.angular, W_0.linear)))

Twist in global frame
[-1.126733    1.05975    -0.779361    1.39733085 -0.58774042 -5.38554104]
Wrench in global frame
[ 1.23911445 -2.47230101 -0.26694735  0.713845   -0.054642   -3.672526  ]
